## 필요한 데이터와 의존성 설치, 모듈 Importing


In [30]:
!kaggle competitions download - c kdtai-2
!unzip kdtai-2.zip - d dataset


zsh:1: command not found: kaggle
unzip:  cannot find or open kdtai-2.zip, kdtai-2.zip.zip or kdtai-2.zip.ZIP.


In [31]:
!pip list - -format = freeze


Package                       Version
----------------------------- ---------
appnope                       0.1.3
asttokens                     2.2.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
certifi                       2022.12.7
charset-normalizer            3.1.0
click                         8.1.3
comm                          0.1.3
debugpy                       1.6.7
decorator                     5.1.1
executing                     1.2.0
filelock                      3.12.0
future                        0.18.3
gensim                        4.3.1
idna                          3.4
importlib-metadata            6.6.0
ipykernel                     6.22.0
ipython                       8.12.0
jamotools                     0.1.10
jedi                          0.18.2
Jinja2                        3.1.2
joblib                        1.2.0
JPype1                        1.4.1
jupyter_client                8.2.0
jupyter_core                  5.3.0
konlpy        

In [32]:
%pip install - r requirements.txt


ERROR: Invalid requirement: '-'
Note: you may need to restart the kernel to use updated packages.


In [56]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader, BatchSampler
import pandas as pd
import os
import nltk
from konlpy.tag import Okt
import jamotools
import re
from gensim.models import FastText
from collections import Counter
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/lee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## FastText Model Training 및 저장


### Load Corpus


In [34]:
# okt = Okt()
# current_path = os.getcwd()
# train_df = pd.read_csv(os.path.join(current_path, "dataset", "train.csv"))
# train_corpus = []
# for sentence in train_df["text"]:
#     train_corpus.append(okt.morphs(sentence, stem=True))
# train_corpus[:3]

### Preprocess Corpus


In [35]:
# okt = Okt()
# train_corpus = okt.morphs(train_corpus, stem=True)
# train_corpus[:5]


In [36]:
# korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
# jamo_splited_corpus = []


# def jamo_split(word):
#     return korean.sub('', jamotools.split_syllables(word))


# for word in train_corpus:
#     jamo_splited_corpus.append(jamo_split(word))

# jamo_splited_corpus[:5]

### Train FastText Model


In [37]:
# model = FastText(train_corpus, vector_size=300,
#                  window=5, min_count=3, workers=4, sg=1)

In [38]:
# model.save("embedding_model/korean_fasttext.bin")


In [39]:
device = torch.device("mps")
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: mps


In [40]:
from enum import Enum
from nltk.tokenize import word_tokenize
from soynlp.hangle import decompose, character_is_korean
import re
from tqdm import tqdm
from konlpy.tag import Okt


class Dataset_type(Enum):
    TRAIN = 0
    TEST = 1


class Korean_dataset(Dataset):
    def __init__(self, file_path, dataset_type: Dataset_type,
                 embedding_model, use_jamo_split=False,
                 exclude_stopwords=False, max_length=None):
        super().__init__()
        self.file_path = file_path
        # self.transform = transform
        self.dataset_type = dataset_type
        self.data_df = pd.read_csv(self.file_path)
        self.embedding_model = embedding_model
        self.word2idx = embedding_model.wv.key_to_index
        self.use_jamo_split = use_jamo_split
        self.exclude_stopwords = exclude_stopwords
        self.max_length = max_length
        current_path = os.getcwd()
        stopwords_path = os.path.join(
            current_path, "utils", "StopWordKorean.csv")
        self.stopwords = pd.read_csv(stopwords_path, index_col=False)
        self.preprocessed_src_list = self._preprocessing(self.data_df)

    def __len__(self):
        return len(self.preprocessed_src_list)

    def __getitem__(self, idx):
        src = self.preprocessed_src_list[idx]
        if self.dataset_type == Dataset_type.TRAIN:
            trg = torch.tensor(self.data_df.loc[idx, "label"]).to(device)
            return src, trg
        else:
            return src

    def _preprocessing(self, df):
        src_list = []
        rows_to_drop = []
        okt = Okt()

        # 중복 제거 (Only Training)
        if self.dataset_type == Dataset_type.TRAIN:
            df.drop_duplicates(subset=['text'], inplace=True)

        # 한글, 영어, 공백, 숫자 빼고 전부 제거
        df["text"] = df["text"].apply(
            lambda x: re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\s]', '', x))

        # null 값 제거
        df.dropna(subset=["text"], inplace=True)
        df.reset_index(drop=True, inplace=True)

        for idx, text in tqdm(enumerate(df.loc[:, "text"])):
            # 형태소 분리
            tokens_per_sentence = okt.morphs(text, stem=True)

            # 불용어 제거 (Optional)
            if self.exclude_stopwords:
                tokens_per_sentence = self._remove_stopwords(
                    tokens_per_sentence)

                if len(tokens_per_sentence) == 0:
                    rows_to_drop.append(idx)
                    continue

            # 자모 분리 (Optional)
            if self.use_jamo_split:
                tokens_per_sentence = self._split_jamo(tokens_per_sentence)

            # fastText 초기 model 기반 정수 인코딩
            encodings_of_tokens = self._indexing(
                tokens_per_sentence, self.max_length, self.embedding_model)

            # subwords 활용을 위한 초기 모델 기반 임베딩 벡터
            vectors_of_tokens = self._embedding(
                tokens_per_sentence, self.max_length, self.embedding_model)

            # src -> preprocessed_src_list[idx] =
            # ([encoding * max_vocab] , [[embeddingvector] * max_vocab]
            src = (encodings_of_tokens, vectors_of_tokens)
            src_list.append(src)

        df.drop(rows_to_drop, inplace=True)
        df.reset_index(drop=True, inplace=True)

        return src_list

    def _remove_stopwords(self, tokens_per_sentence):
        return [token for token in tokens_per_sentence
                if token not in self.stopwords["text"].values]

    def _split_jamo(self, tokens_per_sentence):
        korean = re.compile('[^1!ㄱ-ㅣ가-힣]+')
        return [korean.sub('', jamotools.split_syllables(
            token)) for token in tokens_per_sentence]

    def _indexing(self, tokens_per_sentence, max_length, embedding_model):
        padding_index = 0
        indexes_of_tokens = [padding_index] * max_length
        # 정수 인코딩 + padding
        for idx, token in enumerate(tokens_per_sentence):
            if idx == max_length:
                break
            if token in embedding_model.wv.key_to_index:
                token_id = self.word2idx[token]
            else:
                token_id = 0

            indexes_of_tokens[idx] = token_id
        return torch.tensor(np.array(indexes_of_tokens))

    def _embedding(self, tokens_per_sentence, max_length, embedding_model):
        padding_vector = embedding_model.wv['<pad>']
        vectors_of_tokens = [padding_vector] * max_length
        # 임베딩 + padding
        for idx, token in enumerate(tokens_per_sentence):
            if idx == max_length:
                break
            vectors_of_tokens[idx] = embedding_model.wv[token]
        return torch.tensor(np.array(vectors_of_tokens))


In [41]:
current_path = os.getcwd()
model_file_path = os.path.join(
    current_path, "embedding_model", "korean_fasttext.bin")
embedding_model = FastText.load(model_file_path)


In [42]:
from torch.utils.data import random_split

current_path = os.getcwd()
train_file_path = os.path.join(current_path, "dataset", "train.csv")
test_file_path = os.path.join(current_path, "dataset", "test.csv")

train_set = Korean_dataset(file_path=train_file_path,
                           dataset_type=Dataset_type.TRAIN,
                           use_jamo_split=False,
                           exclude_stopwords=False,
                           embedding_model=embedding_model,
                           max_length=30)
# test_set = Korean_dataset(file_path=test_file_path,
#                           dataset_type=Dataset_type.TEST,
#                           use_jamo_split=False,
#                           exclude_stopwords=False,
#                           embedding_model=embedding_model)


/var/folders/3_/19vyqy_n009677wdd2xcl_xh0000gn/T/ipykernel_1839/1731002018.py:31: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  self.stopwords = pd.read_csv(stopwords_path, index_col=False)
65334it [02:38, 413.16it/s]


In [43]:
train_set.preprocessed_src_list[2000][0]


tensor([6168, 1351,  128, 1344,  878,   72,    1,   94,  137,    0,  132, 3245,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0])

In [44]:
len(train_set)

65334

In [72]:
from torch.nn.utils.rnn import pad_sequence


class LSTM_Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, embedding_model, num_layers=1):
        super(LSTM_Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embedding_model = embedding_model
        self.word_vectors = self.embedding_model.wv
        self.embedding = nn.Embedding.from_pretrained(
            torch.from_numpy(self.word_vectors.vectors), padding_idx=0)
        self.lstm = nn.LSTM(input_dim, hidden_dim,
                            num_layers, batch_first=True, dropout=0.3)
        self.fc1 = nn.Linear(2*input_dim, input_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, encoding, vector_1):
        h0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding.device)
        c0 = torch.zeros(self.num_layers, encoding.size(0),
                         self.hidden_dim).to(encoding
                                             .device)
        vector_2 = self.embedding(encoding)
        x = torch.cat([vector_1, vector_2], dim=2)
        x = self.fc1(x)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        output = out[:, -1, :]
        x = self.fc2(output)
        return x


class LengthSorter(object):
    def __init__(self, dataset):
        self.dataset = dataset

    def __iter__(self):
        indices = list(range(len(self.dataset)))
        indices.sort(key=self._key)
        return iter(indices)

    def __len__(self):
        return len(self.dataset)

    def _key(self, idx):
        return len(self.dataset[idx])


input_dim = 300
hidden_dim = 128
output_dim = 7
num_layers = 4
batch_size = 64
num_of_epoch = 20
learning_rate = 0.0005

model = LSTM_Net(input_dim=input_dim, hidden_dim=hidden_dim,
                 output_dim=output_dim, num_layers=num_layers, embedding_model=embedding_model).to(device)

# 손실 함수와 최적화 알고리즘을 정의합니다.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = CosineAnnealingWarmupRestarts(
    optimizer, first_cycle_steps=4000, cycle_mult=1.0, max_lr=0.001, min_lr=0.0001, warmup_steps=500, gamma=0.9)


def train(dataset, valid_rate, batch_size, optimizer, scheduler, num_of_epoch, valid_random_seed=None, patience=3):

    def collate_fn(batch):
        inputs = [item[0] for item in batch]
        targets = [item[1] for item in batch]

        # 입력 데이터의 길이를 기준으로 내림차순으로 정렬
        sorted_idx = sorted(range(len(inputs)),
                            key=lambda i: len(inputs[i]), reverse=True)
        inputs = [inputs[i] for i in sorted_idx]
        targets = [targets[i] for i in sorted_idx]

        # 패딩된 새로운 텐서 생성
        inputs = pad_sequence(inputs, batch_first=True, padding_value=0.0)
        # print(inputs.shape)

        return inputs, torch.tensor(targets)

    def collate_fn_fixed(batch):
        inputs = [item[0] for item in batch]
        targets = [item[1] for item in batch]

        # 각 시퀀스를 최대 길이 30으로 자르거나 패딩

        inputs = [torch.tensor(seq[:30]) for seq in inputs]

        # 모든 시퀀스를 동일한 길이로 패딩
        inputs = pad_sequence(inputs, batch_first=True, padding_value=0.0)

        return inputs, torch.tensor(targets)

    dataset = dataset
    train_set_count = int(len(dataset) * valid_rate)
    val_set_count = len(dataset) - train_set_count

    if valid_random_seed is not None:
        torch.manual_seed(valid_random_seed)

    train_set, val_set = random_split(
        dataset, [train_set_count, val_set_count])
    train_batch_sampler = BatchSampler(LengthSorter(
        train_set), batch_size=batch_size, drop_last=True)
    val_batch_sampler = BatchSampler(LengthSorter(
        val_set), batch_size=batch_size, drop_last=True)

    # train_loader = DataLoader(train_set, batch_sampler=train_batch_sampler,
    #                           collate_fn=collate_fn)
    # val_loader = DataLoader(val_set, batch_sampler=val_batch_sampler,
    #                         collate_fn=collate_fn)

    train_loader = DataLoader(train_set, batch_size=batch_size,
                              shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size,
                            shuffle=True)

    optimizer = optimizer
    criterion = nn.CrossEntropyLoss().to(device)
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []
    lrs = []
    is_early_stopping = False
    best_val_accuracy = 0

    for epoch in tqdm(range(num_of_epoch)):
        model.train()
        correct_train = 0
        for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
            encoding = data[0]
            vector_1 = data[1]
            encoding, vector_1, target = encoding.to(
                device), vector_1.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(encoding, vector_1)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            lrs.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
            _, pred = torch.max(output.data, 1)
            correct_train += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, lrs: {}'.format(
                    epoch + 1, batch_idx *
                    len(data[0]), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item(), lrs[-1]))

        train_accuracy = 100. * (correct_train / len(train_set))
        train_losses.append(loss.item())
        train_accs.append(train_accuracy)
        print(f"Epoch: {epoch + 1} - train accuracy: {train_accuracy}")

        model.eval()
        correct_val = 0
        with torch.no_grad():
            for batch_idx, (data, target) in tqdm(enumerate(val_loader)):
                encoding = data[0]
                vector_1 = data[1]
                encoding, vector_1, target = encoding.to(
                    device), vector_1.to(device), target.to(device)
                output = model(encoding, vector_1)
                loss = criterion(output, target)
                _, pred = torch.max(output.data, 1)
                correct_val += pred.eq(target.view_as(pred)).sum().item()

            val_accuracy = 100. * (correct_val / len(val_set))
            val_losses.append(loss.item())
            val_accs.append(val_accuracy)

            if val_accuracy > best_val_accuracy:
                current_patience = patience
                best_val_accuracy = val_accuracy
                best_model = model.state_dict()
                torch.save(best_model, 'model.pt')
            else:
                current_patience -= 1
                if current_patience < 0:
                    print("Early Stopping!")
                    is_early_stopping = True

            print('Epoch {} finished: train loss = {}, val loss = {}'.format(epoch + 1,
                                                                             train_losses[-1], val_losses[-1]))
            print(f"Epoch: {epoch + 1} - Validation accuracy: {val_accuracy}")

            if is_early_stopping:
                break

기존 사전학습된 FaxtText 임베딩 모델


주어진 데이터로 만들어진 FastText 임베딩 모델


In [46]:
# train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
#       optimizer=optimizer, num_of_epoch=num_of_epoch, valid_random_seed=42)

In [47]:
# train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
#       optimizer=optimizer, num_of_epoch=num_of_epoch, valid_random_seed=42)

모델 형태 개선


In [54]:
# train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
#       optimizer=optimizer, num_of_epoch=num_of_epoch, valid_random_seed=42)

  0%|          | 0/20 [00:00<?, ?it/s]

Train Epoch: 1 [0/58800 (0%)]	Loss: 1.952313


Train Epoch: 1 [6400/58800 (11%)]	Loss: 1.806568


Train Epoch: 1 [12800/58800 (22%)]	Loss: 1.671595


Train Epoch: 1 [19200/58800 (33%)]	Loss: 1.565732


...

Train Epoch: 20 [57600/58800 (98%)]	Loss: 0.590502


919it [00:31, 28.94it/s]


Epoch: 20 - train accuracy: 84.6156462585034


103it [00:02, 43.68it/s]
100%|██████████| 20/20 [11:14<00:00, 33.71s/it]

Epoch 20 finished: train loss = 0.3393833637237549, val loss = 1.1182928085327148
Epoch: 20 - Validation accuracy: 76.52280379553108


스케쥴러 도입


In [66]:
train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
      optimizer=optimizer, scheduler=scheduler, num_of_epoch=num_of_epoch, valid_random_seed=42)

  0%|          | 0/20 [00:00<?, ?it/s]

Train Epoch: 1 [0/58800 (0%)]	Loss: 1.952313, lrs: 0.0001


Train Epoch: 1 [6400/58800 (11%)]	Loss: 1.797451, lrs: 0.00028


Train Epoch: 1 [12800/58800 (22%)]	Loss: 1.592779, lrs: 0.00045999999999999996


Train Epoch: 1 [19200/58800 (33%)]	Loss: 1.315614, lrs: 0.00064


...

Train Epoch: 20 [57600/58800 (98%)]	Loss: 0.546037, lrs: 0.0003503927533796431


919it [00:30, 29.69it/s]


Epoch: 20 - train accuracy: 82.6156462585034


103it [00:02, 45.31it/s]
100%|██████████| 20/20 [12:02<00:00, 36.12s/it]

Epoch 20 finished: train loss = 0.3772847652435303, val loss = 0.6260377168655396
Epoch: 20 - Validation accuracy: 76.23201714110806


EarlyStopping, BestModel Saving


In [73]:
train(dataset=train_set, valid_rate=0.9, batch_size=batch_size,
      optimizer=optimizer, scheduler=scheduler, num_of_epoch=num_of_epoch, valid_random_seed=42)

  0%|          | 0/20 [00:00<?, ?it/s]

Train Epoch: 1 [0/58800 (0%)]	Loss: 1.934796, lrs: 0.0001


Train Epoch: 1 [6400/58800 (11%)]	Loss: 1.800639, lrs: 0.00028


Train Epoch: 1 [12800/58800 (22%)]	Loss: 1.439610, lrs: 0.00045999999999999996


Train Epoch: 1 [19200/58800 (33%)]	Loss: 1.337596, lrs: 0.00064


...

Train Epoch: 17 [57600/58800 (98%)]	Loss: 0.458803, lrs: 0.00011965926709491925


919it [00:31, 28.86it/s]


Epoch: 17 - train accuracy: 82.00170068027211


103it [00:02, 47.46it/s]
 80%|████████  | 16/20 [09:36<02:24, 36.02s/it]

Early Stopping!
Epoch 17 finished: train loss = 0.4632008969783783, val loss = 1.0022176504135132
Epoch: 17 - Validation accuracy: 77.089072543618
